##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [1]:
!pip install tflite-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 642 kB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 59.9 MB/s 
     |████████████████████████████████| 237 kB 50.8 MB/s 
     |████████████████████████████████| 120 kB 73.1 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 10.9 MB 57.1 MB/s 
     |████████████████████████████████| 3.4 MB 45.3 MB/s 
     |████████████████████████████████| 6.4 MB 43.4 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 840 kB 63.6 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 42.6 MB 1.8 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.2 MB/s 
     |█████████████████████████████

Import the required packages.

In [2]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

Crear la carpeta helmet

In [3]:
#cargar el zip en los archivos
!ls helmet

helmet_photos.zip


In [7]:
!unzip helmet/helmet_photos.zip

Archive:  helmet/helmet_photos.zip
   creating: helmet_photos/
   creating: helmet_photos/without_helmet/
  inflating: helmet_photos/without_helmet/BikesHelmets10.png  
  inflating: helmet_photos/without_helmet/BikesHelmets103.png  
  inflating: helmet_photos/without_helmet/BikesHelmets110.png  
  inflating: helmet_photos/without_helmet/BikesHelmets111.png  
  inflating: helmet_photos/without_helmet/BikesHelmets112.png  
  inflating: helmet_photos/without_helmet/BikesHelmets119.png  
  inflating: helmet_photos/without_helmet/BikesHelmets128.png  
  inflating: helmet_photos/without_helmet/BikesHelmets130.png  
  inflating: helmet_photos/without_helmet/BikesHelmets133.png  
  inflating: helmet_photos/without_helmet/BikesHelmets156.png  
  inflating: helmet_photos/without_helmet/BikesHelmets168.png  
  inflating: helmet_photos/without_helmet/BikesHelmets170.png  
  inflating: helmet_photos/without_helmet/BikesHelmets175.png  
  inflating: helmet_photos/without_helmet/BikesHelmets176.png  

In [ ]:
#no la ocupo
image_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [9]:
data = DataLoader.from_folder('helmet_photos')
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 184, num_label: 2, labels: with_helmet, without_helmet.


2. Customize the TensorFlow model.

In [10]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
5/5 [==============================] - 14s 112ms/step - loss: 0.6501 - accuracy: 0.6500
Epoch 2/5
5/5 [==============================] - 2s 383ms/step - loss: 0.5893 - accuracy: 0.7500

3. Evaluate the model.

In [11]:
loss, accuracy = model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step - loss: 0.5842 - accuracy: 0.7368


4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [12]:
model.export(export_dir='.', with_metadata=False)

INFO:tensorflow:Assets written to: /tmp/tmp_gltwhpk/assets


INFO:tensorflow:Assets written to: /tmp/tmp_gltwhpk/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:Saving labels in ./labels.txt


INFO:tensorflow:Saving labels in ./labels.txt


After this simple 4 steps, we can now download the model and label files, and continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android/#4).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).
